# Generierung von synthetischen Daten

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import itertools
import time
import shutil
import threading
import sys
import h5py
from unrealcv import client
from datetime import datetime

from matplotlib import image
import PIL.Image

## Hilfsfunktionen

In [ ]:
def establish_connection():
    
    client.connect()
    if not client.isconnected():
        status = 'UnrealCV server is not running. Run the game'
    else:
        status = client.request('vget /unrealcv/status')
    
    return status

In [ ]:
def create_new_df():
    
    return pd.DataFrame(data=None, index=None, columns=['Filename', 'Elevation', 'Azimuth'])

In [ ]:
def write_to_csv(df):
    
    if os.path.isfile(_CSV_FILE):
        df.to_csv(_CSV_FILE, mode='a', index=False, header=False)
    else:
        df.to_csv(_CSV_FILE, index=False)

In [ ]:
def create_coordinates(state, e_deg, a_deg, radius):
    
    fully_zenit_enabled = True
    
    if(state=='PLS' and 90 in e_deg):
        e_deg = np.delete(e_deg, np.argwhere(e_deg==90))
        fully_zenit_enabled = False

    e_sin = list(map(lambda e: np.sin(np.deg2rad(e)), e_deg))
    e_cos = list(map(lambda e: np.cos(np.deg2rad(e)), e_deg))
    a_sin = list(map(lambda a: np.sin(np.deg2rad(a)), a_deg))
    a_cos = list(map(lambda a: np.cos(np.deg2rad(a)), a_deg))

    coordinates = {'deg': list(itertools.product(e_deg, a_deg)),
                   'x': [0 if (e==90 or e==-90) else round(radius*e*a, 5) for e in e_cos for a in a_cos],
                   'y': [0 if (e==90 or e==-90) else round(radius*e*a, 5) for e in e_cos for a in a_sin],
                   'z': list(np.repeat([round(radius*e, 5) for e in e_sin], len(a_deg)))}
    
    if not fully_zenit_enabled:
        coordinates['deg'].append((90, 0))
        coordinates['x'].append(0)
        coordinates['y'].append(0)
        coordinates['z'].append(radius)
    
    return coordinates

In [ ]:
def create_csv_dataset(img_format, mesh_name):
    
    counter = pd.read_csv(_CSV_FILE).shape[0] if os.path.isfile(_CSV_FILE) else 0
    df = create_new_df()
    
    for i in range(0, len(_CAM_COORDINATES['deg'])):
        for j in range(0, len(_PLS_COORDINATES['deg'])):
            
            counter = counter + 1
            
            a_respect_c = _PLS_COORDINATES['deg'][j][1] - _CAM_COORDINATES['deg'][i][1]
            
            if _PLS_COORDINATES['deg'][j][0] == 90:
                a_respect_c = 0
            elif a_respect_c < 0:
                a_respect_c = 360 - abs(a_respect_c)                
    
            img_name = '{}_{}_CE_{}_CA_{}_LE_{}_LA_{}.png'.format(
                str(counter).zfill(6),
                mesh_name,
                _CAM_COORDINATES['deg'][i][0],
                _CAM_COORDINATES['deg'][i][1],
                _PLS_COORDINATES['deg'][j][0],
                a_respect_c)
        
            df = df.append(pd.DataFrame(data=[[img_name, _PLS_COORDINATES['deg'][j][0], a_respect_c]],
                                        index=None,
                                        columns=['Filename', 'Elevation', 'Azimuth']))
                    
            img_base = rename_and_convert_image(img_name, img_format)
            delete_image(img_base)
             
            if(i%5000==0) & (i>0):
                print(counter)
                write_to_csv(df)
                df = create_new_df()
                
    write_to_csv(df)
    df = create_new_df()

In [ ]:
def rename_and_convert_image(img_name, img_format):
    
    is_renamed = False
    img_dst = _IMG_DST_PATH + img_name
    
    while(not is_renamed):
        try:                    
            img_base = os.listdir(_IMG_SRC_PATH)[0]
            img_conv = PIL.Image.open(_IMG_SRC_PATH + img_base).convert(img_format)
            img_conv.save(img_dst)
            is_renamed = True
        except:
            #print('Fehler beim Verschieben der Datei')
            continue
            
    return img_base

In [ ]:
def delete_image(img_base):
    
    is_deleted = False
    
    while(not is_deleted):
        try:                    
            os.remove(_IMG_SRC_PATH + img_base)
            is_deleted = True
        except:
            print('Fehler beim Löschen der Datei {}'.format(img_base))
            continue

In [ ]:
def take_screenshot():

    for i in range (0, len(_CAM_COORDINATES['deg'])):
        assert client.request('vset /camera/0/location {} {} {}'.format(
            _CAM_COORDINATES['x'][i],
            _CAM_COORDINATES['y'][i],
            _CAM_COORDINATES['z'][i])) == 'ok', 'Error while setting camera location'
        
        assert client.request('vset /camera/0/rotation {} {} 0'.format(
            360 - _CAM_COORDINATES['deg'][i][0],
            180 + _CAM_COORDINATES['deg'][i][1])) == 'ok', 'Error while setting camera rotation'
        
        time.sleep(.2)

        for i in range(0, len(_PLS_COORDINATES['deg'])):
            assert client.request('vset /object/BP_PLS_Cone_2/location {} {} {}'.format(
            str(_PLS_COORDINATES['x'][i]),
            str(_PLS_COORDINATES['y'][i]),
            str(_PLS_COORDINATES['z'][i]))) == 'ok', 'Error while setting light location'
            
            client.request('vrun shot')

# Generierung Bilddatensatz

In [ ]:
mesh_name = 'Cone' # Bunny, Buddha, Box, Sphere, Cone
img_format = 'RGB'

In [ ]:
startTime = datetime.now()

_PLS_COORDINATES = create_coordinates(state='PLS',
                                      e_deg=np.arange(5, 95, 5),
                                      a_deg=np.arange(0, 360, 5),
                                      radius=200)

_CAM_COORDINATES = create_coordinates(state='CAM',
                                      e_deg=(0, 30, 60, 90),
                                      a_deg=np.arange(0, 360, 45),
                                      #a_deg=np.arange(0, 1, 1), #Sphere, aufgrund Redundanz nur 1x Azimut
                                      radius=60)

_IMG_SRC_PATH = 'D:\\tkroiss\\UE4_Projects\\Dataset_Creation\\Saved\\Screenshots\\Windows\\'
_IMG_SRC_NAME = 'ScreenShot00000.png'
_IMG_SRC_FILE = _IMG_SRC_PATH + _IMG_SRC_NAME
#_IMG_DST_PATH = 'E:\\tkroiss\\Datasets\\Dataset_New\\'

_IMG_DST_PATH = 'D:\\tkroiss\\Datasets\\Dataset_2019-08-13\\'

print(establish_connection())

if os.path.exists(_IMG_SRC_PATH):
    shutil.rmtree(_IMG_SRC_PATH)

_CSV_FILE = _IMG_DST_PATH + 'images.csv'

producer_thread = threading.Thread(target=take_screenshot)
consumer_thread = threading.Thread(target=create_csv_dataset, args=(img_format, mesh_name))    

producer_thread.start()
consumer_thread.start()
producer_thread.join()
consumer_thread.join()

client.disconnect()

print("Time taken:", datetime.now() - startTime)